## CPSC 368 - Databases in Data Science
### Mental Health Quality Discrepancies Between Men and Women in Tech
By Chloe Zandberg and Olivia Lam

In [1]:
import os

dataset_path = "mental_health.sqlite"

if os.path.exists(dataset_path):
    print("Database file exists!")
else:
    print("Database file NOT found!")

Database file exists!


In [2]:
import pandas as pd 
import numpy as np
import sqlite3 as sql
import altair as alt

# connect to SQL database
con = sql.connect("mental_health.sqlite") # create connection object to database

#### Finding the tables in our database
After establishing a connection to the SQL database, we will find all the tables in the database.

In [3]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, con)
print(tables)

       name
0    Answer
1  Question
2    Survey


#### Converting tables to Pandas dataframes
Given that we must create visualizations in Python, we will now convert our tables to Pandas dataframes and do cleaning and EDA in Python before converting back to SQL.

In [4]:
table_dict = {}
for table_name in tables['name'].tolist(): # Replace with actual table name
    table_dict[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", con) 

# Display the first few rows of each table
print(table_dict['Answer'].head())
print(table_dict['Question'].head())
print(table_dict['Survey'].head())

  AnswerText  SurveyID  UserID  QuestionID
0         37      2014       1           1
1         44      2014       2           1
2         32      2014       3           1
3         31      2014       4           1
4         31      2014       5           1
                                        questiontext  questionid
0                                  What is your age?           1
1                               What is your gender?           2
2                       What country do you live in?           3
3  If you live in the United States, which state ...           4
4                             Are you self-employed?           5
   SurveyID                    Description
0      2014  mental health survey for 2014
1      2016  mental health survey for 2016
2      2017  mental health survey for 2017
3      2018  mental health survey for 2018
4      2019  mental health survey for 2019


### Data Cleaning

Before we can create visualizations in Python, we have to make sure our survey data is in a tidy format. This involves performing an inner join on the 'Question' and 'Answer' tables in order to match text reponses to the question asked and then performing a pivot on the resulting data frame in order to ensure our data is in tidy format—i.e. variables (questions asked in the survey) aren't stored as values in a column. We will perform a pivot analogus to ``pivot_wider`` in R.

In [5]:
qna = pd.merge(table_dict['Answer'], table_dict['Question'], left_on='QuestionID', right_on='questionid', how='inner')
qna.drop(columns=['questionid'], inplace=True)
qna

,AnswerText,SurveyID,UserID,QuestionID,questiontext
0,37,2014,1,1,What is your age?
1,44,2014,2,1,What is your age?
2,32,2014,3,1,What is your age?
3,31,2014,4,1,What is your age?
4,31,2014,5,1,What is your age?
...,...,...,...,...,...
236893,Other,2016,2689,117,Which of the following best describes your wor...
236894,Support,2016,2690,117,Which of the following best describes your wor...
236895,Back-end Developer,2016,2691,117,Which of the following best describes your wor...
236896,DevOps/SysAdmin,2016,2692,117,Which of the following best describes your wor...


Here, we successfully performed an inner join to match questions to the answer text as submitted by survery respondents. Now, we will pivot this data frame wider so that the questions asked become column names in our data frame (this is necessary because these are covariates and as such belong as column headers to adhere to tidy data formatting). 

#### Issues we ran into

It seems that, in the year 2016, some user IDs were re-used, as we cannot only pivot wider using UserID as a unique key where 2016 is included in the data. We decided to drop 2016 for this reason. Otherwise, we have multiple responses to same question by multiple users whose ID did not change between responses (we can't even guarantee that it is the same person receiving the same user ID each question - for example, there may be different people assigned to UserID '1' accidentally). This is not specified in the documentation of data. Thus, we remove 2016 so as not to violate any independence of observations assumptions. 

We also faced the problem of the questions having changed in the survey as the years went on. Now, having pivoted wider, we are faced with NaN values where a particular question was not asked to survey respondants in a given year. This is tough, and forces us to find out if there are any questions that were asked every single year.

In [6]:
qna = qna[qna['SurveyID'] != 2016]
qna = qna.pivot(index='UserID', columns='questiontext', values='AnswerText')
qna

questiontext,Any additional notes or comments,Are you openly identified at work as a person with a mental health issue?,Are you self-employed?,Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.,Describe the circumstances of the badly handled or unsupportive response.,Describe the circumstances of the supportive or well handled response.,Describe the conversation with coworkers you had about your mental health including their reactions.,"Describe the conversation you had with your employer about your mental health, including their reactions and what actions were taken to address your mental health issue/questions.",Describe the conversation you had with your previous coworkers about your mental health including their reactions.,"Describe the conversation you had with your previous employer about your mental health, including their reactions and actions taken to address your mental health issue/questions.",...,Would you be willing to discuss a mental health issue with your coworkers?,Would you be willing to discuss a mental health issue with your direct supervisor(s)?,Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),Would you bring up a mental health issue with a potential employer in an interview?,Would you bring up a physical health issue with a potential employer in an interview?,Would you feel comfortable discussing a mental health issue with your coworkers?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you have been willing to discuss your mental health with your direct supervisor(s)?,Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?
UserID,,,,,,,,,,,,,,,,,,,,,
1,-1,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Some of them,Yes,NaN,No,Maybe,NaN,NaN,NaN,NaN,NaN
2,-1,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,NaN,No,No,NaN,NaN,NaN,NaN,NaN
3,-1,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN
4,-1,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Some of them,No,NaN,Maybe,Maybe,NaN,NaN,NaN,NaN,NaN
5,-1,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Some of them,Yes,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214,NaN,0,0,-1,-1,-1,-1,-1,-1,-1,...,NaN,NaN,0,No,NaN,No,No,Physical health,"No, none of my previous supervisors",Physical health
4215,NaN,0,1,IDK,employee was ostracized and employer discussed...,-1,-1,-1,-1,-1,...,NaN,NaN,0,Maybe,NaN,-1,-1,-1,"No, none of my previous supervisors",Physical health
4216,NaN,0,1,sdsdsdsdsdsd,-1,-1,-1,-1,-1,-1,...,NaN,NaN,0,No,NaN,-1,-1,-1,-1,-1


In [7]:
# Get columns with no NaN values
qna_no_nan = qna.columns[qna.isnull().sum() == 0]

# Convert to list
qna_no_nan_list = qna_no_nan.tolist()

print(qna_no_nan_list)

['Are you self-employed?', 'Do you have a family history of mental illness?', 'Does your employer provide mental health benefits as part of healthcare coverage?', 'Have you ever sought treatment for a mental health disorder from a mental health professional?', 'How many employees does your company or organization have?', 'If you live in the United States, which state or territory do you live in?', 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?', 'Is your employer primarily a tech company/organization?', 'What country do you live in?', 'What is your age?', 'What is your gender?', 'Would you bring up a mental health issue with a potential employer in an interview?']


Now we have found all questions that were asked across all years, and we can do an analysis across 2014, 2015, 2017, 2018, and 2019 to see if trends in these responses have shifted. 

In [8]:
qna = qna[qna_no_nan_list]
qna.head()

questiontext,Are you self-employed?,Do you have a family history of mental illness?,Does your employer provide mental health benefits as part of healthcare coverage?,Have you ever sought treatment for a mental health disorder from a mental health professional?,How many employees does your company or organization have?,"If you live in the United States, which state or territory do you live in?",Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,Is your employer primarily a tech company/organization?,What country do you live in?,What is your age?,What is your gender?,Would you bring up a mental health issue with a potential employer in an interview?
UserID,,,,,,,,,,,,
1,-1,No,Yes,1,6-25,Illinois,Yes,1,United States,37,Female,No
2,-1,No,Don't know,0,More than 1000,Indiana,Don't know,0,United States,44,Male,No
3,-1,No,No,0,6-25,-1,Don't know,1,Canada,32,Male,Yes
4,-1,Yes,No,1,26-100,-1,No,1,United Kingdom,31,Male,Maybe
5,-1,No,Yes,0,100-500,Texas,Don't know,1,United States,31,Male,Yes


First, since we only want to analyze those in the tech industry, we will filter our dataframe accordingly. However, we must first check that the survey response convention (1 = yes, 0 = no) remained the same over the years of the survey. We check this below.

In [9]:
qna['Is your employer primarily a tech company/organization?'].unique()

array(['1', '0', '-1'], dtype=object)

It appears we have ``1 = yes``, ``0 = no``, and ``-1 = no response`` as the response convention for this column. This allows us to operate on this column nicely. We will filter for respondents who confirmed yes, that they do work for a tech organization/company.

In [10]:
qna = qna[qna['Is your employer primarily a tech company/organization?'] == '1']

Because our research study has a lot to do with male and female-indentifying gender discrepancies in tech, we must filter the data set accordingly.

In [11]:
qna['What is your gender?'].nunique()

54

In [12]:
qna['What is your gender?'].unique()

array(['Female', 'Male', 'Male-ish', 'Trans-female',
       'something kinda male?', 'queer/she/they', 'non-binary', 'Nah',
       'All', 'Enby', 'fluid', 'Genderqueer', 'Androgyne', 'Agender',
       'male leaning androgynous', 'Female (trans)', 'queer',
       'A little about you', 'p', 'male', 'female', 'Genderfluid', '-1',
       'uhhhhhhhhh fem genderqueer?', 'God King of the Valajar',
       'Nonbinary', 'Non-binary', 'Agender/genderfluid', 'sometimes',
       'Contextual', '\\-', 'Transfeminine', 'None', 'Ostensibly Male',
       'Female/gender non-binary.', 'Trans woman', 'genderfluid',
       'Demiguy', 'none', 'Trans female', 'She/her/they/them', 'Other',
       'SWM', 'NB', 'gender non-conforming woman', 'Non binary',
       'Masculine', 'Cishet male', 'Agender trans woman', 'femmina',
       'Trans man', 'Trans non-binary/genderfluid', 'agender',
       'Non-binary and gender fluid'], dtype=object)

Because there were 76 different responses for gender due to the setup of the questionnaire (open-ended written response), we need to aggregate the responses that are similar (i.e. "female", "Female", "F", "trans woman" etc. are all female-identifying gender identities and need to be grouped). We also want to compare male and female identifying candidates, so, after grouping, we will have to filter for those two gender identities only.

To avoid mis-gendering individuals by using automated semantic matching algorithms to group these responses into male and female, we have made a list by hand of all female and male-indentifying responses based on this list of unique responses above.

In [13]:
female_ids = ['Female', 'Trans-female', 'Trans woman', 'Female (trans)', 'female', 'Woman-identified', 'trans woman', 'Transfeminine', 'Trans female', 'Female-identified']
male_ids = ['Male', 'MALE', 'Cishet male', 'Trans man', 'Masculine']

In [14]:
qna = qna[qna['What is your gender?'].isin(female_ids + male_ids)]
qna

questiontext,Are you self-employed?,Do you have a family history of mental illness?,Does your employer provide mental health benefits as part of healthcare coverage?,Have you ever sought treatment for a mental health disorder from a mental health professional?,How many employees does your company or organization have?,"If you live in the United States, which state or territory do you live in?",Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,Is your employer primarily a tech company/organization?,What country do you live in?,What is your age?,What is your gender?,Would you bring up a mental health issue with a potential employer in an interview?
UserID,,,,,,,,,,,,
1,-1,No,Yes,1,6-25,Illinois,Yes,1,United States,37,Female,No
3,-1,No,No,0,6-25,-1,Don't know,1,Canada,32,Male,Yes
4,-1,Yes,No,1,26-100,-1,No,1,United Kingdom,31,Male,Maybe
5,-1,No,Yes,0,100-500,Texas,Don't know,1,United States,31,Male,Yes
6,-1,Yes,Yes,0,6-25,Tennessee,Don't know,1,United States,33,Male,No
...,...,...,...,...,...,...,...,...,...,...,...,...
4211,0,Yes,Yes,1,100-500,Texas,I don't know,1,United States of America,42,Female,No
4212,0,Yes,Yes,1,26-100,-1,Yes,1,Brazil,26,Female,Maybe
4213,0,No,No,0,6-25,California,I don't know,1,United States of America,31,female,No


### Exploratory Data Analysis (EDA)
Below, we do some basic EDA for our survey data. Because it is mostly text-based reponses, we will have to handle these inputs accordingly.

In [15]:
alt.Chart(qna).mark_bar().encode(
    alt.X('Is your employer primarily a tech company/organization?:N'),
    alt.Y('count()')
).properties(
    width=300,
    height=150)

alt.Chart(...)

We see that the majority of responses in our dataset correspond to those working in primarily tech organizations.

In [22]:
alt.Chart(qna).mark_bar().encode(
    alt.X('What is your age?:Q'),
    alt.Y('count()')
).properties(
    width=300,
    height=150)

alt.Chart(...)

We can see some respondants put in unrealistic ages when responding to this question, perhaps by mistake.

In [17]:
table_dict['Answer'].shape

(236898, 4)

This survey received 236898 non-unique responses.

In [18]:
table_dict['Answer']['UserID'].nunique()

4218

There were 4218 unique individuals who responded to this study across all 5 years.

In [19]:
table_dict['Survey'].shape

(5, 2)

There are 5 unique years during which this survey took place: 2014-2019.

In [20]:
table_dict['Question']['questionid'].nunique()

105

105 unique questions were asked in this survey.